In [1]:
import sys
sys.path.append("..\\methods")

In [2]:
from estnltk import Text

from estnltk_neural.taggers import StanzaSyntaxTagger

from noun_phrase_extraction import create_df

from POS_sequence import get_POS_sequence
from graph_methods import *

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import sklearn

import os
import csv

In [3]:
df = pd.read_csv('100000_random_sentences.csv')

In [4]:
df

,sentence_id,document_id,sentence_start,sentence_end,subcorpus,text
0,135,1,820,905,aja_kr,Järgmise aasta jooksul nägime teineteist kuskil ehk paar korda - mingitel üritustel .
1,312,2,4807,4858,aja_kr,Aivar Mäed võiks väga hästi ka Mesikäpaks kutsuda .
2,621,9,7153,7242,aja_kr,"Tiina Mõis on aga sõnanud , et lapsed teavad , et haridus on tähtsam kui suur taskuraha ."
3,784,11,4721,4761,aja_kr,Ega selleks pole oma tüdrukut tarvis ! ”
4,839,12,1871,1973,aja_kr,"“ Me ikka ei teadvusta endale , kui õige on ütlemine , et igaüks on oma õnne sepp , ” rõhutab Kersti ."
...,...,...,...,...,...,...
99995,21212466,705319,200739,200759,jututoavestlus,gariby: ma ka ei tea
99996,21212538,705319,202890,202905,jututoavestlus,Shad: irw citty
99997,21212577,705319,203928,203976,jututoavestlus,nuux: 'a nagu käisin rootsis iga päev panin pidu
99998,21212644,705319,205639,205651,jututoavestlus,citty: on on


In [5]:
df = sklearn.utils.shuffle(df).reset_index(drop=True)

### 10000 lause suuruse alamosa töötlemine ja salvestamine

In [6]:
subset = df[:10000]

In [7]:
subset

,sentence_id,document_id,sentence_start,sentence_end,subcorpus,text
0,5858921,312319,0,226,aja_EPL,"Ehkki külmakuudega on libedavastast graniidipuru kogunenud nii palju , et kohati on neist teede äärtesse tekkinud juba vallid , ei kavatse tänavapuhastajad sõelmeid ära koristada enne , kui on selge , et rohkem külma ei tule ."
1,20676731,705250,65236,65274,jututoavestlus,Defender-18: Sokrates korras sellega ?
2,11085184,487500,227,375,aja_pm,"Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul on hõivatud projektiga , mis uurib naiste haridust ja naisi kultuuris ."
3,1331107,51516,5147,5245,aja_EPL,"Kulutasin raha pesupulbrile jms ning kokku olen kulutanud juba 352 krooni , s.t 27 krooni päevas ."
4,6134757,327274,6490,6606,aja_EPL,"Mul pole näiteks sellist tungi , et vaadata ära kõik Hollywoodi või ukraina filmid , aga eesti filmid ilmtingimata ."
...,...,...,...,...,...,...
9995,14053379,619197,197,379,aja_sloleht,"Sel laupäeval eetrisse minevas Raadio 2 saates « Ülekuulamine » sarjab Värnik EOKd ja lisaks ka kettaheitjat Gerd Kanterit , kes polevat mitte keegi , kuid käib ainult raha lunimas ."
9996,1414189,55755,557,567,aja_EPL,ja Intel .
9997,1808883,76437,763,847,aja_EPL,"Kõik ülejäänud kommentaarid on ainult kellegi isiklik arvamus , "" teatas Samaranch ."
9998,8262548,396490,5476,5574,aja_ee,"Istun Pekingi slummis ja piidlen koka suunas , kes oma hurtsikus mulle mingit sööki kokku keerab ."


In [8]:
stanza_tagger = stanza_tagger = StanzaSyntaxTagger(input_type='morph_analysis', input_morph_layer='morph_analysis',
                                   add_parent_and_children=True)

In [10]:
processed_rows = 0
for i, row in subset.iterrows():
    text = Text(row['text']).tag_layer('morph_analysis')
    text.meta['sentence_id'] = row['sentence_id']
    text.meta['document_id'] = row['document_id']
    text.meta['sentence_startend'] = tuple([row['sentence_start'], row['sentence_end']])
    text.meta['subcorpus'] = row['subcorpus']
    stanza_tagger.tag(text)
    subset_df = create_df(text)
    processed_rows+=1

print(f"Processed {processed_rows} rows")
subset_df.to_pickle('data_subset_10000.pkl')


Processed 10000 rows


In [11]:
subset_df = pd.read_pickle('data_subset_10000.pkl')

In [12]:
subset_df

,phrase,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,has_timex_entity
0,Text(text='külmakuudega'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(6, 18)",0,0
1,Text(text='neist teede äärtesse'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(84, 104)",0,0
2,Text(text='libedavastast graniidipuru'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(22, 48)",0,0
3,Text(text='juba vallid'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(114, 125)",0,0
4,Text(text='tänavapuhastajad'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(139, 155)",0,0
...,...,...,...,...,...,...,...,...,...,...
44990,Text(text='mis'),2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nsubj_phrase,"(59, 62)",0,0
44991,"Text(text='kategooria , mis määrab edaspidiseks ka koorijuhi palgamäära')",2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obj_phrase,"(46, 106)",0,0
44992,Text(text='ka koorijuhi palgamäära'),2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obj_phrase,"(83, 106)",0,0
44993,Text(text='Laulupeo'),2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nmod_phrase,"(0, 8)",0,0


### Markdown-failid 10000 lause suurusest alamosast

In [86]:
subset_df = pd.read_pickle('data_subset_10000.pkl')

In [87]:
subset_df

,phrase,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,has_timex_entity
0,Text(text='külmakuudega'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(6, 18)",0,0
1,Text(text='neist teede äärtesse'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(84, 104)",0,0
2,Text(text='libedavastast graniidipuru'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(22, 48)",0,0
3,Text(text='juba vallid'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(114, 125)",0,0
4,Text(text='tänavapuhastajad'),2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(139, 155)",0,0
...,...,...,...,...,...,...,...,...,...,...
44990,Text(text='mis'),2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nsubj_phrase,"(59, 62)",0,0
44991,"Text(text='kategooria , mis määrab edaspidiseks ka koorijuhi palgamäära')",2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obj_phrase,"(46, 106)",0,0
44992,Text(text='ka koorijuhi palgamäära'),2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obj_phrase,"(83, 106)",0,0
44993,Text(text='Laulupeo'),2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nmod_phrase,"(0, 8)",0,0


In [89]:
all_pos = []
for phrase in subset_df['phrase']:
    all_pos.append(get_POS_sequence(phrase))
subset_df.insert(len(subset_df.columns), 'pos_sequence', all_pos)

In [91]:
all_graphs = []
for phrase in subset_df['phrase']:
    graph = create_graph(phrase)
    all_graphs.append(graph)
subset_df.insert(len(subset_df.columns), 'graph', all_graphs)

In [92]:
subset_df['graph_code'] = [get_graph_code(graph) for graph in subset_df['graph']]
subset_df['graph_code_pos'] = [get_graph_code(graph, ['pos']) for graph in subset_df['graph']]
subset_df['graph_code_pos_ner_timex'] = [get_graph_code(graph, ['pos', 'ner_timex']) for graph in subset_df['graph']]

In [93]:
subset_df.to_pickle('data_subset_10000.pkl')

In [96]:
phrase_lengths = [len(subset_df['phrase'][i].words) for i in range(len(subset_df))]

subset_df.insert(1, 'phrase_length', phrase_lengths) 

In [97]:
subset_df

,phrase,phrase_length,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,has_timex_entity,pos_sequence,graph,graph_code,graph_code_pos,graph_code_pos_ner_timex
0,Text(text='külmakuudega'),1,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(6, 18)",0,0,S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))"
1,Text(text='neist teede äärtesse'),3,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(84, 104)",0,0,P-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, det),(3, 2, nmod))","(S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))","(0-0-0,S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))"
2,Text(text='libedavastast graniidipuru'),2,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(22, 48)",0,0,A-S,"(1, 2, 0)","((0, 2, root),(2, 1, amod))","(S-A,(0, 2, root),(2, 1, amod))","(0-0,S-A,(0, 2, root),(2, 1, amod))"
3,Text(text='juba vallid'),2,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(114, 125)",0,0,D-S,"(1, 2, 0)","((0, 2, root),(2, 1, advmod))","(S-D,(0, 2, root),(2, 1, advmod))","(0-0,S-D,(0, 2, root),(2, 1, advmod))"
4,Text(text='tänavapuhastajad'),1,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(139, 155)",0,0,S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44990,Text(text='mis'),1,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nsubj_phrase,"(59, 62)",0,0,P,"(1, 0)","((0, 1, root))","(P,(0, 1, root))","(0,P,(0, 1, root))"
44991,"Text(text='kategooria , mis määrab edaspidiseks ka koorijuhi palgamäära')",8,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obj_phrase,"(46, 106)",0,0,S-Z-P-V-A-D-S-S,"(1, 0, 2, 4, 3, 5, 6, 8, 7)","((0, 1, root),(1, 4, acl:relcl),(4, 2, punct),(4, 3, nsubj),(4, 5, xcomp),(4, 8, obj),(8, 6, advmod),(8, 7, nmod))","(S-V-Z-P-A-S-D-S,(0, 1, root),(1, 4, acl:relcl),(4, 2, punct),(4, 3, nsubj),(4, 5, xcomp),(4, 8, obj),(8, 6, advmod),(8, 7, nmod))","(0-0-0-0-0-0-0-0,S-V-Z-P-A-S-D-S,(0, 1, root),(1, 4, acl:relcl),(4, 2, punct),(4, 3, nsubj),(4, 5, xcomp),(4, 8, obj),(8, 6, advmod),(8, 7, nmod))"
44992,Text(text='ka koorijuhi palgamäära'),3,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obj_phrase,"(83, 106)",0,0,D-S-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, advmod),(3, 2, nmod))","(S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))","(0-0-0,S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))"
44993,Text(text='Laulupeo'),1,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nmod_phrase,"(0, 8)",0,0,H,"(1, 0)","((0, 1, root))","(H,(0, 1, root))","(0,H,(0, 1, root))"


In [98]:
grouped_by_length = subset_df.groupby('phrase_length')

In [99]:
grouped_by_length.size().reset_index(name='count').sort_values(['count'], ascending=False)

,phrase_length,count
0,1,20276
1,2,10623
2,3,4835
3,4,2500
4,5,1555
...,...,...
50,51,1
49,50,1
48,49,1
47,48,1


In [113]:
# Andmete salvestamine pickle-formaati (graafid jäävad alles)
directory = 'example_phrases'

if not os.path.isdir(directory):
    os.mkdir(directory)

# -- target directory path
directory_path = 'example_phrases/'

for name, group in grouped_by_length:
    sub_directory = f'length_{name}'
    os.makedirs(f'{directory_path}{sub_directory}')
    group.to_pickle(f'{directory_path}{sub_directory}/phrases_len_{name}.pkl')

In [145]:
def df_to_markdown(df):
    markdown = ''
    for _, row in df.iterrows():
        markdown += f"## {row['pos_sequence']}\n\nExample phrase: *{row['example_phrase']}*\n\nFrequency: {row['count']}\n\n% of all phrases: {row['%_all_phrases']}\n\n% of POS-sequences with same length: {row['%_pos_sequences_same_length']}\n\nNumber of unique syntax trees (POS-tags): {row['n_unique_trees_pos']}\n\nNumber of unique syntax trees (POS-tags and NER/TIMEX): {row['n_unique_trees_pos_ner_timex']}\n\n% of trees occurring once (POS-tags): {row['%_trees_pos_occurring_once']}\n\n% of trees occurring once (POS-tags and NER/TIMEX): {row['%_trees_pos_ner_timex_occurring_once']}\n\n"
    return markdown


In [146]:
n_all_phrases = len(subset_df)

In [147]:
path = 'example_phrases/'

for subdir in os.listdir(path):
    subdir = subdir+'/'
    file_path = os.path.join(path, subdir)
    for filename in os.listdir(file_path):
        full_filename = os.path.join(file_path, filename)
        
        df = pd.read_pickle(full_filename)
        grouped_df = df.groupby('pos_sequence')
        
        # finding 5 most common POS-sequences
        sorted_groups = grouped_df.size().reset_index(name='count').sort_values(['count'], ascending=False)
        top5_sequences = dict()
        idx = 0
        for i, row in sorted_groups.iterrows():
            top5_sequences[row['pos_sequence']] = row['count']
            idx+=1
            if idx == 5:
                break
        
        # total number of phrases in current length group
        n_phrases_current_len = len(df)
        
        # creating data
        data = {'pos_sequence': [], 'example_phrase': [], 'count': [], '%_all_phrases': [], 
            '%_pos_sequences_same_length': [], 'n_unique_trees_pos': [], 
            'n_unique_trees_pos_ner_timex': [], '%_trees_pos_occurring_once': [], 
            '%_trees_pos_ner_timex_occurring_once': []}

        for name, group in grouped_df:
            if name in top5_sequences.keys():
                group = sklearn.utils.shuffle(group).reset_index(drop=True)
                data['pos_sequence'].append(name)
                data['example_phrase'].append(group['phrase'][0].text)
                data['count'].append(top5_sequences[name])
                data['%_all_phrases'].append(round((top5_sequences[name]/n_all_phrases)*100, 2))
                data['%_pos_sequences_same_length'].append(round((top5_sequences[name]/n_phrases_current_len)*100, 2))
        
                data['n_unique_trees_pos'].append(len(group['graph_code_pos'].unique()))
                data['n_unique_trees_pos_ner_timex'].append(len(group['graph_code_pos_ner_timex'].unique()))
        
        
                n_all_trees_pos = len(group['graph_code_pos'])
                counts_pos = group['graph_code_pos'].value_counts().tolist()
                n_single_occurrences_pos = 0
                for count in counts_pos:
                    if count == 1:
                        n_single_occurrences_pos+=1
                
                n_all_trees_pos_ner_timex = len(group['graph_code_pos_ner_timex'])
                n_single_occurrences_pos_ner_timex = 0
                counts_pos_ner_timex = group['graph_code_pos_ner_timex'].value_counts().tolist()
                for count in counts_pos_ner_timex:
                    if count == 1:
                        n_single_occurrences_pos_ner_timex+=1
        
                data['%_trees_pos_occurring_once'].append(round((n_single_occurrences_pos/n_all_trees_pos)*100, 2))
                data['%_trees_pos_ner_timex_occurring_once'].append(round((n_single_occurrences_pos_ner_timex/n_all_trees_pos_ner_timex)*100, 2))
        
        data_df = pd.DataFrame(data)
        data_df = data_df.sort_values('count', ascending=False)
        md_content = df_to_markdown(data_df)
        with open(f'{file_path}/phrase_examples.md', 'w') as f:
            f.write(md_content)